# 4. Metaprogramming

Julia has extensive support of *metaprogramming*, that is, writing programs that generate other programs.

Julia expressions are actually Julia objects:

In [1]:
ex = :(sin(x)+2) # : quotes the expression; can also use "quote" blocks

:(sin(x) + 2)

In [2]:
typeof(ex)

Expr

In [3]:
ex.args

3-element Array{Any,1}:
  :+       
  :(sin(x))
 2         

*Macros* are functions that are applied to expressions that in turn generate other expressions. This is then evaluated

In [4]:
macro mytime(ex)
    quote
        start = time()
        result = $ex
        stop = time()
        println("time: $(stop-start)")
        result
    end
end

In [5]:
@mytime randn(100,100)*randn(100,100)

time: 0.336475133895874


100x100 Array{Float64,2}:
   7.72472   -14.3001    -16.9959    …   12.8474    -14.461      8.95708 
  20.8511     -7.51564   -23.9604       -17.1929    -11.1617     3.43598 
 -14.3937     -6.77555   -10.7198         5.17217    11.1341     3.09725 
   9.30111     7.60315    16.1225        -9.19503    13.9317     1.28764 
 -17.8591     -1.42329    14.3127        -5.07546    12.6713    -0.13409 
  -7.06515   -10.8714    -19.7221    …    3.03264    -8.54072   -6.12143 
  -7.06014   -16.6314      9.35889       11.4512      2.71412    5.49121 
   1.37385    -3.38068     2.43165      -12.6005     -6.39555    6.05296 
  -5.25616    -0.81159    13.2049         1.17888    23.6282   -15.7371  
   3.94413   -10.2881      0.62758        9.82849     2.54196    8.55863 
   6.09697    -1.56714     2.29207   …  -15.2402     -8.26491   -2.76765 
  -6.32338     3.51781   -12.437         -5.41805     7.76032   -1.26709 
   8.75663    -7.34429     7.41944       -7.28665   -13.8579   -14.2513  
   ⋮        

Julia actually provides its own better `@time` macro:

In [6]:
@time randn(100,100)*randn(100,100)

100x100 Array{Float64,2}:
   7.36264    -5.16124    18.057     …   -2.3356    -10.4512      0.165903
  -6.14068     9.71974    -4.32236        1.32647     0.893908   -8.11686 
   3.28929     1.87309    -9.27867      -13.7467     -1.86668    -4.86062 
   7.61304     1.98826   -25.5106         3.9299     16.471      10.9223  
 -13.4652      1.62118    -6.10506      -11.8424     10.0018      1.66526 
  10.0503     -3.75963    -7.44916   …    5.91258     7.94444    10.2287  
   4.02165     4.4828     21.5808         3.50921   -11.1794      4.8188  
  -8.35946   -11.2699     11.8912       -15.7424      8.46971    21.2659  
  -2.67029    -6.74865    -6.00476       -5.01066    -7.57555     2.76123 
  10.7131      2.25174     3.14949       11.0037      3.86552     0.28695 
   7.77808    -9.73809   -12.2528    …   -3.29164    20.2487      7.12755 
  -3.10852    -3.64188    -9.86743        3.61898     7.24032    13.4142  
   6.00907     0.389224  -10.2726        -2.89122   -12.7742     -3.40151 

Metaprogramming is extremely powerful, but a bit of a dark art (for example, you have to be careful of *hygiene*, that is, not modifying symbols defined elsewhere).

I find the `macroexpand` function useful to figure out what is going on:

In [7]:
macroexpand(:(@mytime sin(x)))

quote  # In[4], line 3:
    #56#start = time() # In[4], line 4:
    #57#result = sin(x) # In[4], line 5:
    #58#stop = time() # In[4], line 6:
    println("time: $(#58#stop - #56#start)") # In[4], line 7:
    #57#result
end

> **Exercise**: write a macro that prints the values of the inputs to a function, and returns the result e.g.
> 
> `@printinput sin(x)`
> 
> should print the value of `x`, then return the result of `sin(x)`.

Some particularly useful macros included in Julia:
* `@time`, `@elapsed` for evaluating performance
* `@printf` for formatted printing
* `@code_warntype` for finding type unstable functions
* `@label`/`@goto` for when you wish you were using Fortran (or just want to translate code).

In [2]:
@printf "%.60f" 0.1

0.100000000000000005551115123125782702118158340454101562500000

In [8]:
function harmonicmean1(X)
    s = 0
    for x in X
        s += 1/x
    end
    1/s
end

0.000407 seconds (155 allocations: 244.745 KB)


harmonicmean1 (generic function with 1 method)

In [9]:
X = rand(1_000_000)
@code_warntype harmonicmean1(X)

Variables:
  X::Array{Float64,1}
  s::ANY
  #s41::Int64
  x::Float64

Body:
  begin  # In[8], line 2:
      s = 0 # In[8], line 3:
      #s41 = 1
      GenSym(2) = (Base

In [10]:
@time harmonicmean1(X)
@time harmonicmean1(X)

.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
      2: 
      GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s41::Int64)::Float64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))
      x = GenSym(4)
      #s41 = GenSym(5) # In[8], line 4:
      s = s::UNION{FLOAT64,INT64} + (Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))::Float64
      3: 
      GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(3),1))::Bool)))) goto 2
      1: 
      0:  # In[8], line 6:
      return 1 / s::UNION{FLOAT64,INT64}::Float64
  end::Float64
  

7.205045769493674e-8

In [11]:
function harmonicmean2(X)
    s = 0.0
    for x in X
        s += 1/x
    end
    1/s
end

0.057201 seconds (2.00 M allocations: 30.529 MB, 14.43% gc time)
  0.036930 seconds (2.00 M allocations: 30.518 MB, 10.99% gc time)


harmonicmean2 (generic function with 1 method)

In [12]:
@code_warntype harmonicmean2(X)

Variables:


In [13]:
@time harmonicmean2(X)
@time harmonicmean2(X)

7.205045769493674e-8

## What happens under the hood

1. Parsing: syntax is parsed (see `parse` function)
 - Macros are expanded
2. Code lowering: this is a simplified syntax (see `@code_lowered`)
 - `a += a` is changed to `a = a + b`
 - loops are replaced with gotos
3. Type inference (see `@code_typed`)
 - tries to infer the type of each object
 - decides which method should be dispatched
4. LLVM code generation (`@code_llvm`)
 - generates LLVM IR (intermediate representation) code
 - some initial optimisation passes 
5. Machine code (`@code_native`)
 - machine-specific optimisations
 - assembly code generation





In [14]:
@code_lowered harmonicmean2(X)

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:X], Any[Any[Any[:X,:Any,0],Any[:s,:Any,2],Any[symbol("#s41"),:Any,2],Any[:x,:Any,18]],Any[],2,Any[]], :(begin  # In[11], line 2:
        s = 0.0 # In[11], line 3:
        GenSym(0) = X
        #s41 = (top(start))(GenSym(0))
        unless (top(!))((top(done))(GenSym(0),#s41)) goto 1
        2: 
        GenSym(1) = (top(next))(GenSym(0),#s41)
        x = (top(getfield))(GenSym(1),1)
        #s41 = (top(getfield))(GenSym(1),2) # In[11], line 4:
        s = s + 1 / x
        3: 
        unless (top(!))((top(!))((top(done))(GenSym(0),#s41))) goto 2
        1: 
        0:  # In[11], line 6:
        return 1 / s
    end))))

In [15]:
@code_typed harmonicmean2(X)

  X::Array{Float64,1}
  s::Float64
  #s41::Int64
  x::Float64

Body:
  begin  # In[11], line 2:
      s = 0.0 # In[11], line 3:
      #s41 = 1
      GenSym(2) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
      2: 
      GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s41::Int64)::Float64
      GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))
      x = GenSym(4)
      #s41 = GenSym(5) # In[11], line 4:
      s = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))))
      3: 
      GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
      unless (Base.box)(Base.Bool,(Base.not_int)((Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(3),1))::Bool)))) goto 2
      1: 
      0:  # In[11], line 6:
      retu

1-element Array{Any,1}:
 :($(Expr(:lambda, Any[:X], Any[Any[Any[:X,Array{Float64,1},0],Any[:s,Float64,2],Any[symbol("#s41"),Int64,2],Any[:x,Float64,18]],Any[],Any[Array{Float64,1},Tuple{Float64,Int64},Int64,Int64,Float64,Int64],Any[]], :(begin  # In[11], line 2:
        s = 0.0 # In[11], line 3:
        #s41 = 1
        GenSym(2) = (Base.arraylen)(X::Array{Float64,1})::Int64
        unless (Base.box)(Base.Bool,(Base.not_int)(#s41::Int64 === (Base.box)(Base.Int,(Base.add_int)(GenSym(2),1))::Bool)) goto 1
        2: 
        GenSym(4) = (Base.arrayref)(X::Array{Float64,1},#s41::Int64)::Float64
        GenSym(5) = (Base.box)(Base.Int,(Base.add_int)(#s41::Int64,1))
        x = GenSym(4)
        #s41 = GenSym(5) # In[11], line 4:
        s = (Base.box)(Base.Float64,(Base.add_float)(s::Float64,(Base.box)(Base.Float64,(Base.div_float)((Base.box)(Float64,(Base.sitofp)(Float64,1)),x::Float64))))
        3: 
        GenSym(3) = (Base.arraylen)(X::Array{Float64,1})::Int64
        unless (Base.box

In [16]:
@code_llvm harmonicmean2(X)


define double @julia_harmonicmean2_21939(%jl_value_t*) {
top:
  %1 = getelementptr inbounds %jl_value_t* %0, i64 1
  %2 = bitcast %jl_value_t* %1 to i64*
  %3 = load i64* %2, align 8
  %4 = icmp eq i64 %3, 0
  br i1 %4, label %L3, label %L.preheader

L.preheader:                                      ; preds = %top
  %5 = load i64* %2, align 8
  %6 = bitcast %jl_value_t* %0 to i8**
  br label %L

L:                                                ; preds = %idxend, %L.preheader
  %"#s41.0" = phi i64 [ %14, %idxend ], [ 1, %L.preheader ]
  %s.0 = phi double [ %16, %idxend ], [ 0.000000e+00, %L.preheader ]
  %7 = add i64 %"#s41.0", -1
  %8 = icmp ult i64 %7, %5
  br i1 %8, label %idxend, label %oob

oob:                                              ; preds = %L
  %9 = alloca i64, align 8
  store i64 %"#s41.0", i64* %9, align 8
  call void @jl_bounds_error_ints(%jl_value_t* %0, i64* %9, i64 1)
  unreachable

idxend:                                           ; preds = %L
  %10 = load i8** %

In [17]:
@code_native harmonicmean2(X)

	.section	__TEXT,__text,regular,pure_instructions
Filename: In[11]
Source line: 3
	pushq	%rbp
	movq	%rsp, %rbp
	xorps	%xmm1, %xmm1
Source line: 3
	cmpq	$0, 8(%rdi)
	je	L98
	movq	8(%rdi), %r8
	movq	%r8, %r9
	negq	%r9
	xorps	%xmm1, %xmm1
	xorl	%edx, %edx
	movabsq	$13233328464, %rax      ## imm = 0x314C49150
	movsd	(%rax), %xmm0
	xorl	%esi, %esi
L49:	cmpq	%r8, %rdx
	jae	L121
Source line: 4
	leaq	(,%rsi,8), %rcx
Source line: 3
	movq	(%rdi), %rax
Source line: 4
	subq	%rcx, %rax
	movaps	%xmm0, %xmm2
	divsd	(%rax), %xmm2
	incq	%rdx
	decq	%rsi
	cmpq	%rsi, %r9
	addsd	%xmm2, %xmm1
	jne	L49
L98:	movabsq	$13233328464, %rax      ## imm = 0x314C49150
	movsd	(%rax), %xmm0
Source line: 6
	divsd	%xmm1, %xmm0
	movq	%rbp, %rsp
	popq	%rbp
	ret
L121:	movl	$1, %eax
Source line: 3
	subq	%rsi, %rax
	movq	%rsp, %rcx
	leaq	-16(%rcx), %rsi
	movq	%rsi, %rsp
	movq	%rax, -16(%rcx)
	movabsq	$jl_bounds_error_ints, %rax
	movl	$1, %edx
	callq	*%rax


## Other metaprogramming tools

`eval` evaluates an `Expr` object:

In [5]:
x = 1
eval(:(sin(x)))

0.8414709848078965

Useful for code generation: e.g. when generating wrappers around C library functions for different types (e.g. see [BLAS](https://github.com/JuliaLang/julia/blob/master/base/linalg/blas.jl) or [LAPACK](https://github.com/JuliaLang/julia/blob/master/base/linalg/lapack.jl) code).

> **Warning**: if you start writing macros, at some point you will be annoyed that macros don't have access to type information. You may be tempted to use `eval` inside a macro to get around this: *don't* do it, this can cause all sorts of problems (e.g. if you use the macro inside a function).

One way around this is using *staged functions*: these allow on-demand code generation of functions, and have access to input types.